## 1. Install ain-py

Install the ain-py package to work with AI Network.

nest_asyncio is installed to solve errors that occur during asynchronous operation in a notebook environment.

In [1]:
!pip install ain-py
!pip install nest_asyncio

## 2. Import packages

In [2]:
import asyncio
import nest_asyncio
from time import time
from ain.ain import Ain
from ain.types import ValueOnlyTransactionInput

nest_asyncio.apply()

loop = asyncio.get_event_loop()

## 3. Create AI Network wallet

Create AI Network wallet and set default wallet.

In [3]:
ain = Ain("https://testnet-api.ainetwork.ai", chainId=None)

# Create address and Set default account
accounts = ain.wallet.create(1)
my_address = accounts[0]
ain.wallet.setDefaultAccount(my_address)

## 4. Create App

Make sure your app's name includes a timestamp to avoid overlapping app names.

After setting app_path, set the owner in the path of /manage_app/{appName}/create/{key} using the SET_VALUE operation.

If Owner is set, it has all owner permissions(branch_owner, write_function, write_owner, write_rule) on the path /apps/{appName}.

A transaction occurs during this process.

In [4]:
# Use your own app name
app_name = f"my_app_{int(time()*1000)}"
app_path = f"/apps/{app_name}"

# Create an app at /apps/{appName}. With the admin config below,
# only 'my_address' will have owner & write permissions at /apps/{app_name}.
async def create_my_app():
    result = await ain.db.ref(
        f'/manage_app/{app_name}/create/{int(time()*1000)}'
    ).setValue(
        ValueOnlyTransactionInput(
            value={
                'admin': {
                    my_address: True
                }
            },
            nonce=-1
        )
    )
    print(result)
loop.run_until_complete(create_my_app())

{'tx_hash': '0x30a9b45c57e3dc58edcefbe10ed42ad3bb0f2d4e0f488a784e893f2f88d3bc74', 'result': {'gas_amount_total': {'bandwidth': {'service': 2, 'app': {'my_app_1641880649312': 2}}, 'state': {'service': 1566}}, 'gas_cost_total': 0, 'func_results': {'_createApp': {'op_results': {'0': {'path': '/apps/my_app_1641880649312', 'result': {'code': 0, 'bandwidth_gas_amount': 1}}, '1': {'path': '/apps/my_app_1641880649312', 'result': {'code': 0, 'bandwidth_gas_amount': 1}}, '2': {'path': '/manage_app/my_app_1641880649312/config', 'result': {'code': 0, 'bandwidth_gas_amount': 1}}}, 'code': 0, 'bandwidth_gas_amount': 0}}, 'code': 0, 'bandwidth_gas_amount': 1, 'gas_amount_charged': 1568}}


## 5. Send Transaction

Write the value "Hello AI Network" using SET_VALUE to the /apps/{appName} sub-path.

A transaction occurs during this process.

In [5]:
# Write value to user_message_path.
async def set_value():
    user_message_path = f'{app_path}/messages/{my_address}'
    result = await ain.db.ref(
        f'{user_message_path}/{int(time()*1000)}/user'
    ).setValue(
        ValueOnlyTransactionInput(value='Hello AI Network', nonce=-1)
    )
    print(result)
loop.run_until_complete(set_value())

{'tx_hash': '0x920bc14c3cf2f1faf994b478911d4d022b05e42b823124dcacb57fcdcc68e74d', 'result': {'gas_amount_total': {'bandwidth': {'service': 0, 'app': {'my_app_1641880649312': 1}}, 'state': {'service': 0, 'app': {'my_app_1641880649312': 966}}}, 'gas_cost_total': 0, 'code': 0, 'bandwidth_gas_amount': 1, 'gas_amount_charged': 0}}
